# Julia 機器學習：DecisionTree 決策樹

## 作業 030：乳癌預測資料集

請使用隨機森林模型建立一個分類模型，來預測乳癌資料集中，為良性或是惡性的腫瘤。

In [31]:
using DecisionTree, RDatasets, DataFrames, MLDataUtils, Statistics

## 讀取資料

In [32]:
biopsy = dataset("MASS", "biopsy")

699×11 DataFrame. Omitted printing of 3 columns
│ Row │ ID      │ V1    │ V2    │ V3    │ V4    │ V5    │ V6     │ V7    │
│     │ String  │ Int32 │ Int32 │ Int32 │ Int32 │ Int32 │ Int32⍰ │ Int32 │
├─────┼─────────┼───────┼───────┼───────┼───────┼───────┼────────┼───────┤
│ 1   │ 1000025 │ 5     │ 1     │ 1     │ 1     │ 2     │ 1      │ 3     │
│ 2   │ 1002945 │ 5     │ 4     │ 4     │ 5     │ 7     │ 10     │ 3     │
│ 3   │ 1015425 │ 3     │ 1     │ 1     │ 1     │ 2     │ 2      │ 3     │
│ 4   │ 1016277 │ 6     │ 8     │ 8     │ 1     │ 3     │ 4      │ 3     │
│ 5   │ 1017023 │ 4     │ 1     │ 1     │ 3     │ 2     │ 1      │ 3     │
│ 6   │ 1017122 │ 8     │ 10    │ 10    │ 8     │ 7     │ 10     │ 9     │
│ 7   │ 1018099 │ 1     │ 1     │ 1     │ 1     │ 2     │ 10     │ 3     │
│ 8   │ 1018561 │ 2     │ 1     │ 2     │ 1     │ 2     │ 1      │ 3     │
│ 9   │ 1033078 │ 2     │ 1     │ 1     │ 1     │ 2     │ 1      │ 1     │
│ 10  │ 1033078 │ 4     │ 2     │ 1     │ 1     │ 2     │ 1      │ 2     │
⋮
│ 689 │ 603148  │ 4     │ 1     │ 1     │ 1     │ 2     │ 1      │ 1     │
│ 690 │ 654546  │ 1     │ 1     │ 1     │ 1     │ 2     │ 1      │ 1     │
│ 691 │ 654546  │ 1     │ 1     │ 1     │ 3     │ 2     │ 1      │ 1     │
│ 692 │ 695091  │ 5     │ 10    │ 10    │ 5     │ 4     │ 5      │ 4     │
│ 693 │ 714039  │ 3     │ 1     │ 1     │ 1     │ 2     │ 1      │ 1     │
│ 694 │ 763235  │ 3     │ 1     │ 1     │ 1     │ 2     │ 1      │ 2     │
│ 695 │ 776715  │ 3     │ 1     │ 1     │ 1     │ 3     │ 2      │ 1     │
│ 696 │ 841769  │ 2     │ 1     │ 1     │ 1     │ 2     │ 1      │ 1     │
│ 697 │ 888820  │ 5     │ 10    │ 10    │ 3     │ 7     │ 3      │ 8     │
│ 698 │ 897471  │ 4     │ 8     │ 6     │ 4     │ 3     │ 4      │ 10    │
│ 699 │ 897471  │ 4     │ 8     │ 8     │ 5     │ 4     │ 5      │ 10    │

In [33]:
describe(biopsy)

11×8 DataFrame. Omitted printing of 1 columns
│ Row │ variable │ mean    │ min     │ median │ max       │ nunique │ nmissing │
│     │ Symbol   │ Union…  │ Any     │ Union… │ Any       │ Union…  │ Union…   │
├─────┼──────────┼─────────┼─────────┼────────┼───────────┼─────────┼──────────┤
│ 1   │ ID       │         │ 1000025 │        │ 95719     │ 645     │          │
│ 2   │ V1       │ 4.41774 │ 1       │ 4.0    │ 10        │         │          │
│ 3   │ V2       │ 3.13448 │ 1       │ 1.0    │ 10        │         │          │
│ 4   │ V3       │ 3.20744 │ 1       │ 1.0    │ 10        │         │          │
│ 5   │ V4       │ 2.80687 │ 1       │ 1.0    │ 10        │         │          │
│ 6   │ V5       │ 3.21602 │ 1       │ 2.0    │ 10        │         │          │
│ 7   │ V6       │ 3.54466 │ 1       │ 1.0    │ 10        │         │ 16       │
│ 8   │ V7       │ 3.43777 │ 1       │ 3.0    │ 10        │         │          │
│ 9   │ V8       │ 2.86695 │ 1       │ 1.0    │ 10        │         │          │
│ 10  │ V9       │ 1.58941 │ 1       │ 1.0    │ 10        │         │          │
│ 11  │ Class    │         │ benign  │        │ malignant │ 2       │          │

In [34]:
using StatsBase

In [35]:
v6 = biopsy[!, :V6]
freq = countmap(v6)
delete!(freq, missing)
println(freq)
most_freq_v6 = first(keys(sort(freq, by = last, rev=true)))
biopsy[!, :V6] = coalesce.(v6, most_freq_v6)

Dict{Union{Missing, Int32},Int64}(2 => 30,7 => 8,9 => 9,10 => 132,8 => 21,6 => 4,4 => 19,3 => 28,5 => 30,1 => 402)


699-element Array{Int32,1}:
  1
 10
  2
  4
  1
 10
 10
  1
  1
  1
  1
  1
  3
  ⋮
  1
  1
  1
  1
  5
  1
  1
  2
  1
  3
  4
  5

In [36]:
describe(biopsy)

11×8 DataFrame. Omitted printing of 1 columns
│ Row │ variable │ mean    │ min     │ median │ max       │ nunique │ nmissing │
│     │ Symbol   │ Union…  │ Any     │ Union… │ Any       │ Union…  │ Nothing  │
├─────┼──────────┼─────────┼─────────┼────────┼───────────┼─────────┼──────────┤
│ 1   │ ID       │         │ 1000025 │        │ 95719     │ 645     │          │
│ 2   │ V1       │ 4.41774 │ 1       │ 4.0    │ 10        │         │          │
│ 3   │ V2       │ 3.13448 │ 1       │ 1.0    │ 10        │         │          │
│ 4   │ V3       │ 3.20744 │ 1       │ 1.0    │ 10        │         │          │
│ 5   │ V4       │ 2.80687 │ 1       │ 1.0    │ 10        │         │          │
│ 6   │ V5       │ 3.21602 │ 1       │ 2.0    │ 10        │         │          │
│ 7   │ V6       │ 3.69242 │ 1       │ 1.0    │ 10        │         │          │
│ 8   │ V7       │ 3.43777 │ 1       │ 3.0    │ 10        │         │          │
│ 9   │ V8       │ 2.86695 │ 1       │ 1.0    │ 10        │         │          │
│ 10  │ V9       │ 1.58941 │ 1       │ 1.0    │ 10        │         │          │
│ 11  │ Class    │         │ benign  │        │ malignant │ 2       │          │

In [37]:
rfModel = DecisionTree.RandomForestClassifier()

RandomForestClassifier
n_trees:             10
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             nothing
ensemble:            nothing

In [44]:
features = Matrix(float.(biopsy[!, 2:10]))
labels = Vector{String}(biopsy[!, :Class])

699-element Array{String,1}:
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 ⋮
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "benign"
 "benign"
 "benign"
 "benign"
 "malignant"
 "malignant"
 "malignant"

In [49]:
#DecisionTree.fit!(rfModel, features, labels)
#model = build_forest(labels, features)
accuracy = nfoldCV_forest(labels, features, 5)

2×2 Array{Int64,2}:
 92   0
  2  45

2×2 Array{Int64,2}:
 91   2
  0  46

2×2 Array{Int64,2}:
 88   1
  0  50

2×2 Array{Int64,2}:
 93   1
  0  45

2×2 Array{Int64,2}:
 87   1
  0  51


Fold 1
Classes:  ["benign", "malignant"]
Matrix:   
Accuracy: 0.9856115107913669
Kappa:    0.9675157747137181

Fold 2
Classes:  ["benign", "malignant"]
Matrix:   
Accuracy: 0.9856115107913669
Kappa:    0.9678612716763005

Fold 3
Classes:  ["benign", "malignant"]
Matrix:   
Accuracy: 0.9928057553956835
Kappa:    0.9844501622105382

Fold 4
Classes:  ["benign", "malignant"]
Matrix:   
Accuracy: 0.9928057553956835
Kappa:    0.9836643553884123

Fold 5
Classes:  ["benign", "malignant"]
Matrix:   
Accuracy: 0.9928057553956835
Kappa:    0.984577832020415

Mean Accuracy: 0.9899280575539569


5-element Array{Float64,1}:
 0.9856115107913669
 0.9856115107913669
 0.9928057553956835
 0.9928057553956835
 0.9928057553956835